<a href="https://colab.research.google.com/github/GimenesPaula/GimenesPaula/blob/main/Projeto%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fazer Upload planilhas
1.   Launches
2.   Inci
3.   Sales
4.   Project

In [ ]:
#Launches
from google.colab import files
uploaded = files.upload()

Saving Q1-2023-Cosmeticos.xlsx to Q1-2023-Cosmeticos.xlsx


In [ ]:
filename = next(iter(uploaded))

In [ ]:
# INCI name produtos
from google.colab import files
inci = files.upload()

Saving inci.xlsx to inci.xlsx


In [ ]:
filename2 = next(iter(inci))

In [ ]:
#Projetos Distribuidores
from google.colab import files
proj = files.upload()

Saving Projetos.xlsx to Projetos.xlsx


In [ ]:
filename3 = next(iter(proj))

In [ ]:
#Vendas Distribuidores
from google.colab import files
dist = files.upload()

Saving Vendas.xlsx to Vendas.xlsx


In [ ]:
filename4 = next(iter(dist))

# Bibliotecas Phyton

In [ ]:
#Carrega Bibliotecas
import pandas as pd
import numpy as np
import re

In [ ]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 14.4 MB/s eta 0:00:00


In [ ]:
from unidecode import unidecode

##Carrega tabela do Mintel

In [ ]:
#Carrega o banco de dados como tabela
data = '/content/'+filename
df_launches = pd.read_excel(data)
pd.options.display.max_columns = None
df_launches.nunique()

Número do Produto               1136
Data de Publicação                64
Produto                          807
Marca                            811
Empresa                          347
Mercado                            1
Categoria                          4
Sub-Categoria                     36
Descrição do Produto            1136
Tipo de Lançamento                 5
Fabricante                       330
Local de fabricação               20
Manufacturer Company Address     339
Ingredients (Standard form)     1112
Preço em moeda local             510
Link da Imagem Primária         1136
Animal de Estimação                0
Conveniência                      20
Menos                              0
Funcional                         58
Natural                           20
Mais                               2
Posicionamento                    10
Adequado para                      4
Demográfico                       10
Sem                               41
Realce de Beleza                  90
É

##Carrega Tabela de INCI

In [ ]:
data = '/content/'+filename2
df_inci = pd.read_excel(data)
pd.options.display.max_columns=None
df_inci.nunique()

Produto        50
Ingrediente    46
Prioridade      3
dtype: int64

##Carrega Tabela de Projetos

In [ ]:
data = '/content/'+filename3
df_proj = pd.read_excel(data)
pd.options.display.max_columns=None
df_proj.nunique()

Distribuidor      3
Customer        913
UF               16
Product         144
Fase             17
dtype: int64

## Carrega Planilha de Vendas do Distribuidor

In [ ]:
data = '/content/'+filename4
df_dist = pd.read_excel(data)
pd.options.display.max_columns=None
df_dist.nunique()

Customer(at distrib)        891
Country, Customer             1
Unnamed: 2                    1
Distributor                   4
Product management            3
Material (at WAC)            55
Unnamed: 6                   54
Market segment                1
Unnamed: 8                    1
Application                   6
Month last sale|Timeline     12
2022                        346
dtype: int64

# Define funções

In [ ]:
#this checks if any combination of INCI as present in Ingredient
def verifica_ingrediente(formula,produto,material):
  quantidade = len(produto.difference(formula))
  if quantidade == 0:
    return material
  return None

In [ ]:
#If last code is true, this returns the material name
def procura_produtos(formula, produtos, materiais):
  formula = formula.copy()
  resultados = []
  for prod, mat in zip(produtos, materiais):
    resultado = verifica_ingrediente(formula, prod, mat)
    if resultado is not None:
      formula = formula.difference(prod) ## Para remover os ingredientes já encontrados numa nova busca.
      resultados.append(resultado)
  return resultados

In [ ]:
#this code transpose data. Used when we bring each category and the number of lauches.
def transpor (df, coluna, linha):
  for cat in df[coluna].unique():
    f = df[coluna] == cat
    df[cat] = df[f][linha]
    f = df[cat].isna()
    df.loc[f, cat] = df.loc[f, cat].apply(lambda x:[])

In [ ]:
#Code to solve an issue from algoritms
def to_set(x):
    try:
        if isinstance(x, pd.DataFrame):
            return x.apply(to_set)
        else:
            return set(x.sum())
    except:
        pass

In [ ]:
#ode to solve an issue from algoritms
def to_set_2(x):
    try:
        if isinstance(x, pd.DataFrame):
            return x.apply(to_set_2)
        else:
            return list(set(x))
    except:
        pass

In [ ]:
#This is a dictionary of words that may be removed from Company, Manufacture
#in order to group manufacture despite of spelling mistakes
def formatar_fabricantes(coluna):
  remover = [
             'fabrica', 'fabricacao', 'manuf', 'tercerizacao', 'manufacturing',
             'farmacia', 'farmacias', 'farmaceuticos', 'drogaria','drogarias', 'farm',
             'cosmetico', 'cometico','cosmeticos', 'cometicos', 'cosmetics',
             'cosmiatria', 'cosmetica', 'cosmeticas', 'cosm', 'cosme', 'kosmetics',
             'ltda','epp', 'eireli', 'me', 'eirell', 'sa', 'eir', 'ltd',
             'indústria','industria', 'industrias', 'industrial', 'indutrial', 'ind', 'industriae',
             'comercio', 'comecio', 'comercial', 'corporation', 'com',
             'imp', 'exp',  'prod', 'exportacao','importacao','importadora', 'import',
             'do', 'e', 'para', 'de', 'le', 'la',
             'do brasil', 'aerossois', 'aerosol', 'aerossol', 'tecnologia',
             'laboratorio','laboratorios', 'laboratories', 'laboratoire', 'instituto', 'group',
             'pesquisa', 'desenvolvimento', 'naturais', 'professional', 'aromatizantes',
             'produtos', 'Produto', 'saneantes', 'quimica', 'quimicos', 'quimico', 'farmaceutica', 'veterinário',
             'higiene','pessoal', 'limpeza', 'beleza', 'botanico',
             'perfumes','fragrancias', 'perfumaria', 'medicamentos', 'perf',
             'distribuidora', 'dist',
             'international', 'internacional', 'sulamericana', 'nacional',
             'nbsp;', 'veer','serv', 'vajerista',
           ]
  pat = '\\b({})\\b'.format('|'.join(remover))
  coluna = (
    coluna
    .apply(unidecode)
    .str.lower()
    .replace('\.', ' ',regex=True)
    .replace('\/', '',regex=True)
    .replace('\-', '',regex=True)
    .replace(pat,'',regex=True)
    .replace(',', '',regex=True)
    .replace('\s+','',regex=True) #Tirar numeros
    .str.strip()
  )
  return coluna

In [ ]:
#Dis dict clean the name of material
def formatar_material(coluna):
  remover = ['BELSIL®', 'BELSIL', 'BELSIM',
             'DRUM', 'ST', 'PE', 'CANISTER', 'RD', 'CARTON', 'IBC PE', '2', 'WOPAL',
             'SBPAL', 'STPAL', 'HOBBOCK', 'SACK', 'BOTTLE',
             '0,45KG','10KG','15KG','25KG', '30G', '60G', '100KG', '140KG', '150KG', '160KG',
             '180KG', '190KG', '200KG', '210KG', '900KG', '950KG', '190',
             'KG', 'AMOSTRA', 'PA', 'XY', 'CP', 'SILICONE ELASTOMER GEL',
             'BR', 'DE', ' E', 'BLOCKED',
             'EMB. 0,100 KG', 'Ver  Pf 100', 'USP',
           ]
  pat = '\\b({})\\b'.format('|'.join(remover))
  coluna = (
    coluna
    .replace(pat,'',regex=True)
    .replace('\#','', regex=True)
    .replace('\"','', regex=True)
    .str.replace('\/', '',regex=True)
    .str.replace('\-', '',regex=True)
    .str.replace('\*', '',regex=True)
    .str.title()
    .str.strip()
  )
  return coluna

# Mintel

## Define Estados


In [ ]:
dict = {'AC':'AC', 'AL':'AL', 'AM':'AM', 'AP':'AP', 'BA':'BA', 'CE':'CE', 'DF':'DF', 'ES':'ES', 'GO':'GO', 'MA':'MA',
     'MG':'MG', 'MS':'MS', 'MT':'MT', 'PA':'PA', 'PB':'PB', 'PE':'PE', 'PI':'PI', 'PR':'PR', 'RJ':'RJ', 'RN':'RN',
     'RO':'RO', 'RR':'RR', 'RS':'RS', 'SC':'SC', 'SE':'SE', 'SP':'SP', 'TO':'TO', 'Sao Paulo':'SP', 'São Paulo':'SP',
     'Sao Paolo':'SP','Osasco': 'SP', 'Mogi das Cruzes': 'SP', 'Cotia':'SP', 'Diadema':'SP', 'São Roque':'SP',
     'São Sebastião':'SP', 'Lins':'SP', 'Cajamar':'SP', 'Pirassununga':'SP', 'Paulista': 'SP', 'São Paolo':'SP',
     'Elisario':'SP', 'Valinhos':'SP', 'Piracaia':'SP', 'São José do Rio Preto':'SP', 'Sao José do Rio Preto':'SP',
     'São José dos Campos':'SP',      'Araras':'SP', 'São Bernardo do Campo':'SP', 'São José dos Campos':'SP',
     'Sao Caetano do Sul':'SP', 'São José do Rio Preto':'SP', 'Brodowski':'SP', 'Cajuru':'SP', 'Itaquaquecetuba':'SP',
     'Itupeva':'SP', 'SO':'SP', 'Mairiporã':'SP', 'Itaquera':'SP', 'Franco da Rocha':'SP', 'Guarulhos':'SP',
     'Bairro Jardim America':'SP', 'Jardim Brazil':'SP', 'Agudos':'SP', 'Itapira':'SP', 'Rib. Preto':'SP',
     'Rio de Janeiro':'RJ', 'Rio De Janeiro':'RJ', 'Nova Iguaçu': 'RJ', 'Nova Iguacu': 'RJ', 'Nivopolis':'RJ',
     'Duque de Caixas':'RJ', 'Engenheiro Pedreira':'RJ',
     'Vila Velha':'ES', 'Marechal Floriano':'ES',
     'Belo Horizonte':'MG', 'Minas Gerais':'MG', 'Pocos de Caldas':'MG', 'Viçosa':'MG', 'Vicosa':'MG',
     'Pouso Alegre': 'MG', 'Uberlândia':'MG', 'Uberlandia':'MG', 'Ipatinga':'MG', 'Mirabela':'MG','Nova Lima':'MG',
     'São José da Lapa':'MG', 'Sao José da Lapa':'MG', 'Sao Jose da Lapa':'MG',
     'Porto Alegre':'RS', 'Estância Velha': 'RS', 'Caxias do Sul':'RS', 'Passo Fundo':'RS', 'Gravatai':'RS',
     'Santa Catarina':'SC', 'Joinville':'SC','Camboriu':'SC', 'Maluche':'SC', 'Araquari':'SC', 'Criciúma': 'SC',
     'Criciuma': 'SC',
     'Colombo':'PR', 'Paraná':'PR', 'Curitiba':'PR', 'Sao José dos Pinhais':'PR','BR 116 Km 85':'PR', 'Cambé':'PR',
     'Goiana':'GO', 'Goiânia':'GO', 'Anápolis':'GO', 'Anapolis':'GO', 'Brasilia': 'DF', 'Go':'GO',
     'Rondônia':'RO','Novo Hamburgo':'RO','Manaus':'AM', 'Pará':'PA',
     'Bahia':'BA', 'Lauro de Freites': 'BA', 'Av. Queira Deus':'BA', 'Vila Cia Sul':'BA',
     'Recife':'PE', 'Camaragibe': 'PE', 'Fortaleza':'CE', 'Ceará':'CE'}

In [ ]:
df_launches['State'] = df_launches['Manufacturer Company Address'].str.extract('('+'|'.join(dict)+')',expand=False).map(dict)
#Exibe a quantidade de casos únicos
df_launches['State'].value_counts(dropna=False)
#Em output se filtra estados vazios

SP     574
NaN    220
MG      69
RJ      60
GO      56
PR      48
RS      40
SC      16
CE      14
BA      13
PE      11
ES       6
PI       3
DF       2
SE       2
AM       1
PA       1
Name: Estados, dtype: int64

## Remover Clientes  da planilha Mintel


In [ ]:
dictClientes = {'Devintex Cosméticos':'1', 'Devintex':'1','Procosa Produtos de Beleza':'1', 'Colgate-Palmolive':'1', 'Unilever':'1',
                'Savoy Indústria de Cosméticos':'1', 'Beiersdorf':'1', 'Johnson & Johnson':'1',
                'Procter & Gamble':'1','Química Amparo':'1', "Oréal":'1', 'Procosa':'1',
                'Unilever Brasil Industrial':'1', 'Botica Comercial Farmacêutica':'1'}
dictMarcas = { 'Boticário':'1', 'Siàge':'1', 'Natura ':'1', 'Avon ':'1', 'Eudora':'1',
              'Wella':'1', 'The Body Shop':'1', 'Quem Disse,':'1', 'Salon Line':'1', 'Maybelline':'1'}
df_launches['Clientes']= df_launches['Manufacturer'].str.extract('('+'|'.join(dictClientes)+')',expand=False).map(dictClientes)
df_launches['Clientes2'] = df_launches['Marca'].str.extract('('+'|'.join(dictMarcas)+')',expand=False).map(dictMarcas)
df_ = df_launches[df_launches['Clientes']=='1']
df_ = df_[df_['Clientes2']=='1']
df_.to_excel("LancClientes.xlsx")

In [ ]:
#Remove clientes  da base de tratamento
df_launches = df_launches[df_launches['Clientes']!='1']
df_launches = df_launches[df_launches['Clientes2']!='1']

## Search dictOTHERS words in the ingredient, in order to identify cases where the words occure.

In [ ]:
#Dicionário Silicones geral
dictOTHERS = {'methicone':'1', 'Dimethicone':'1','methicone Crosspolymer':'1','methiconol':'1',
              'ylsiloxysilicate':'1', 'ylsilsesquioxane':'1', 'Disiloxane':'1', 'Silica':'1','siloxane':'1'}

In [ ]:
df_launches['Silicone'] = df_launches['Ingredients (On pack)'].str.extract('('+'|'.join(dictOTHERS)+')',expand=False).map(dictOTHERS)

##Search "Free From"


In [ ]:
dictSil = {'Silicone':'1'}

In [ ]:
df_launches['Free from'] = df_launches['Free from'].str.extract('('+'|'.join(dictSil)+')',expand=False).map(dictSil)

## Search ingredients and identify products that mache INCI



In [ ]:
df_inci.dropna(inplace=True)
df_inci['Product'] = formatar_material(df_inci['Product'])

In [ ]:
#Cria uma lista iterável dos ingredientes nos Produtos
df_inci['Ing'] = formatar_material(df_inci['Ingrediente']).str.split(', ').apply(set)

In [ ]:
#Conta a quantidade de ingredientes
df_inci.sort_values('Prioridade', ascending=True, inplace=True)

In [ ]:
 #Dicionário de palavras a remover da coluna Ingredientes no Mintel
 dictIng={' Andor ':',',
          ', ':',',
          ' ,':',',
          ' (And) ':',',
          }

In [ ]:
#Cria uma lista iterável das formulas do mintel
df_launches['Ingredients (On pack)'] = df_launches['Ingredients (On pack)'].astype(str)
df_launches['Ing'] = formatar_material(df_launches['Ingredients (On pack)']).replace(dictIng,regex=True).dropna().str.split(',').apply(set)

In [ ]:
#Aplica a função Procura Produtos
df_launches[''] = df_launches['Ing'].apply(lambda formulacao:procura_produtos(formulacao, df_inci['Ing'], df_inci['Product']))

## Cria Indice para junção

In [ ]:
df_launches.set_index(['Record ID'])
df_launches['Date Published'] = pd.to_datetime(df_launches['Date Published'], format='%Y-%m-%d')
df_launches['Quarter'] = df_launches['Date Published'].dt.quarter
df_launches['Year'] = df_launches['Date Published'].dt.year

In [ ]:
#Cria a coluna que irá indexar a mistura com a planilha do mintel
df_launches['Manufacturer']=df_launches['Manufacturer'].astype(str)
df_launches['KeyManuf'] = formatar_fabricantes(df_launches['Manufacturer'])

## Baixa relatório Linha a Linha, com analise de ingrediente

In [ ]:
df_launches.columns

Index(['Número do Produto', 'Data de Publicação', 'Produto', 'Marca',
       'Empresa', 'Mercado', 'Categoria', 'Sub-Categoria',
       'Descrição do Produto', 'Tipo de Lançamento', 'Fabricante',
       'Local de fabricação', 'Manufacturer Company Address',
       'Ingredients (Standard form)', 'Preço em moeda local',
       'Link da Imagem Primária', 'Animal de Estimação', 'Conveniência',
       'Menos', 'Funcional', 'Natural', 'Mais', 'Posicionamento',
       'Adequado para', 'Demográfico', 'Sem', 'Realce de Beleza',
       'Ético e ambiental', 'Produto testado', 'Estados', 'Clientes',
       'Clientes2', 'Territorio', 'Silicone', 'Claim Sem Silicone', 'Ing',
       'WACKER', 'Trimestre', 'Ano', 'Cliente Resumo', 'Marca Resumo'],
      dtype='object')

In [ ]:
col=['Quarter','Sub-Category', 'Product', 'Brand', 'Company', 'Manufacturer', 'State', 'Free from', 'Price in local currency',
     'Silicone', '', 'Ing', 'Manufacturer Company Address',]

In [ ]:
df_launches[col].to_excel("Lancamentos Linha a Linha.xlsx")

## Baixar relatório agrupar por Fabricante

In [ ]:
transpor(df_launches, 'Categoria', '')

In [ ]:
df_launches.rename(columns={'Skincare':'Skin', 'Hair Products':'Hair',
                          'Soap & Bath Products':'Bath', 'Record ID': 'Total Lançamentos'},
                 inplace=True)

In [ ]:
df_a = df_launches.groupby(['KeyManuf', 'State'])[['Hair', 'Skin', 'Colour Cosmetics', 'Bath']].apply(to_set)

In [ ]:
df_b = df_launches.groupby(['KeyManuf', 'State']).agg({
                                            'Total Lançamentos': 'count',
                                             'Silicone':'count',
                                             'Free from':'count',
                                             })

In [ ]:
df_launches_fab = pd.concat([df_a, df_b], axis=1).reset_index()

In [ ]:
df_launches_fab.to_excel('Manufacturer.xlsx')

# Vendas

In [ ]:
#Garantir que apenas serão considerados os clientes com vendas
df_dist['Year'].dropna(inplace=True)

In [ ]:
df_dist['Material']= formatar_material(df_dist['Material'])
df_dist['Material'] = df_dist['Material'].astype(str)

In [ ]:
df_dist['KeyManuf'] = formatar_fabricantes(df_dist['Customer(at distrib)'])

In [ ]:
#agrupo os produtos em uma lista, por cliente
df_dist_unic = df_dist.groupby(['KeyManuf','Distributor']).agg({'Year':'sum','Material':lambda x:list(set(x))}).reset_index()

In [ ]:
df_dist_unic['Material'] = df_dist_unic['Material'].apply(set)
df_dist_unic.rename(columns={'Material': 'Sales'}, inplace=True)

In [ ]:
df_dist_unic.drop_duplicates(subset=['KeyManuf', 'Distributor'], keep='first', inplace=True)

# Projetos

In [ ]:
df_proj['Product']= formatar_material(df_proj['Product'])

In [ ]:
df_proj['KeyManuf'] = formatar_fabricantes(df_proj['Customer'])

In [ ]:
#Dicionário Silicones geral
dictFASE = {'APROVADO':'APROVADO','Closed Won': 'APROVADO', 'FINALIZADO': 'APROVADO',
            'CANCELADO':'CANCELADO', 'CANCELADO/REPROVADO': 'CANCELADO', 'STAND BY':'CANCELADO',
            'REPROVADO':'REPROVADO', 'Closed Lost': 'REPROVADO',
            'EM ANDAMENTO':'EM ANDAMENTO', 'EM TESTE': 'EM ANDAMENTO', 'Negotiation/Review': 'EM ANDAMENTO',
            'Proposal/Price Quote':'EM ANDAMENTO', 'Prospecting':'EM ANDAMENTO',
            'Qualification':'EM ANDAMENTO',
            }

In [ ]:
df_proj['FASE'] = df_proj['Fase'].str.strip().str.extract('('+'|'.join(dictFASE)+')',expand=False).map(dictFASE)

In [ ]:
df_proj_unic = df_proj.groupby(['KeyManuf','UF','Distribuidor','FASE']).agg({
                                                          'Product': lambda x:list(set(x)),
                                                          }).reset_index()

In [ ]:
transpor(df_proj_unic, 'FASE', 'Product')

In [ ]:
df_proj_unic_2 = df_proj_unic.groupby(['KeyManuf', 'UF', 'Distribuidor'])[['CANCELADO', 'EM ANDAMENTO', 'APROVADO', 'REPROVADO']].apply(to_set).reset_index()

In [ ]:
df_proj_unic_2.head(2).to_excel('Projetos&Status.xlsx')

,Cliente Resumo,UF,Distribuidor,CANCELADO,EM ANDAMENTO,APROVADO,REPROVADO
0,2k,SC,EMBACAPS,{Adm 8301},{},{},{}
1,a3eventosservicos,SC,EMBACAPS,{},{Eg 5},{},{}


# Agrupar Projeto e Vendas

In [ ]:
df_proj_vend = df_proj_unic_2.merge(df_dist_unic, on=['KeyManuf', 'Distribuidor'], how='left')
df_vend_proj = df_dist_unic.merge(df_proj_unic_2, on=['KeyManuf', 'Distribuidor'], how='left')
df_temp = df_proj_vend.merge(df_vend_proj, on=['KeyManuf', 'Distribuidor'], how='left')
df_temp = df_temp.dropduplicate()

In [ ]:
df_temp.to_excel('Projetos ou Vendas.xlsx')

# Agrupar Mintel e Vendas e Projeto

In [ ]:
df_launches_vend_proj = df_launches_fab.merge(df_vend_proj, on='KeyManuf', how='left')

In [ ]:
df_launches_vend_proj.rename(columns={'': ' Prod Poss','Customer(at distrib)':'Customer Sales','Distributor':'Dist. Sales',
                                    'Sales': 'Prod Sales','Year': 'Sales Volume', 'Customer': 'Customer Proj Fab',
                                    'Distribuidor':'Dist Proj Fab', 'Project':'Prod Proj Fab',
                                    'Fase':'Fase Proj Fab',
                                         },
                                inplace=True)
#df_launches_vend_proj.drop_duplicates()
df_launches_vend_proj.sample(2)

,Cliente Resumo,Trimestre,Estados,Cabelo,Pele,Maquilagem,Banho,Total Lançamentos,Silicone,Claim Sem Silicone,Dist Proj Fab,Sales Volume,Prod Sales,UF,CANCELADO,EM ANDAMENTO,APROVADO,REPROVADO
110,gcferreira,1,SP,NaN,NaN,NaN,NaN,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,naturelle,1,SP,{Cm 740},"{Elastomero, Gb 150, Hdk N20, Cm 740}",{},{},10,9,0,FOCUS QUIMICA,0.785,"{Adm 8105, Dm 350, Pdm 20, Cm 1000, Adm 9000}",SP,{},{Gb 7030},{Eg 5},{}


In [ ]:
df_launches_vend_proj.to_excel('Lançamentos Vendas Projeto.xlsx')